# Introdução

Uma rede de supermercados (normalmente WallMart) usando mineração de dados descobre que há uma estranha correlacão entre compra de cerveja e compra de fraldas (veja mais sobre isso [aqui](https://www.theregister.com/2006/08/15/beer_diapers/)). Em algumas versões a rede coloca um estande de cerveja ao lado das fraldas.

As técnicas de mineração de regras de associação e de conjunto de intens (itemset) frequentes é que permitem tirar este tipo de conclusão.

Esse notebook surge das minhas anotações pessoais do curso [Pattern Discovery in Data Mining](https://www.coursera.org/learn/data-patterns) junto com materiais complementares (vídeos, artigos, etc) que usei para me ajudar a entender o conteúdo apresentado no curso. Algumas das imagens usadas veem dos slides do curso que mencionei.

# Possibilidades do uso de Regras de Associação

Neste tipo de problema há um conjunto de itens (itens num supermercado) e há transações que contem um subconjunto dos itens (uma compra). Mas podemos abstrair isso e generalizar o uso desse recurso. Seguem algumas possibilidades:

* os itens podem ser paginas num site, a transação as paginas visitadas em diferentes interações com o site.
* o conceito de transação pode não ser localizado no tempo. Pode ser uma pessoa e os itens podem ser aplicativos que essa pessoa instalou no seu celular (não necessariamente ao mesmo tempo).

* pode ser proteinas ativas em diferentes tecidos de diferentes individuos (uma transação é a combinação de tecido e individuo).

# Itemsets frequentes

Itemsets frequentes são conjunto de itens que aparecem juntos em pelo menos *s*% das transações. O número *s*, que precisa ser fornecido para o algoritmo é chamado de suporte.

Vamos assumir as seguintes transações:

* ABC
* AC
* CD
* AB
* BD
* D

Se o suporte foi definido como 1/3, ou seja queremos conjuntos de itens que aparecem em
pelo menos 2 das 6 transações, então AC, AB e D são itemsets frequentes.

Um padrão muito grande de intemset contém uma possibilidade de combinações muito grande. Suponha que tenhamos os seguintes itemsets:

* $T_1= {a_1, ..., a_{50}}$
* $T_2= {a_1, ..., a_{50}, ..., a_{100}}$

As possibilidades de combinações de sub-padrões destes itemsets seriam:

$(\sum_{k=1}^{100}{100\choose k}) - 1= (1 + 1)^{100} -1 = 2^{100} - 1 $

Para encontrar todos os intemsets frequentes, seria necessário computar todas essas possibilidades, o que é impossível de fazer em tempo útil. Portando temos algumas outras definições de itemsets que ajudam a diminuir esse número de possibilidades.

![](imgs/1.png)


Segue as definições de itemsets fechados e maximais:

* um itemset *i* é maximal se ele tem suporte maior que *s* e todos os itemsets que incluem *i* tem suporte menor que s, ou seja, não há super padrão Y frequente em que i é sub-conjunto. No exemplo anterior voltar AB, AC e D são maximais. Essa abordagem perde informações pois sabemos apenas que i é frequente mas não o real suporte do mesmo.


* Um itemset *i* é fechado (closed) se todos os itemsets que o incluem tem suporte menor que *i*.

# A Propriedade de "Fechamento para baixo" dos itemsets frequentes

O Algoritmo Apriori assume que:

"*Se um itemset é frequente, todos seus subitemsets também o serão*"

Isso faz sentido, suponha que o itemset $T_1= {a_1, ..., a_{50}}$ seja frequente, então  ${(a_1), (a_1, a_2), ...}$ também serão frequentes. 

Seguindo esse princípio, deixamos nosso processo de calcular itemsets frequentes mais rápido dado que não precisamos nos preocupar de obter o suporte dos subitemsets de um itemset frequente.

# Apriori: a abordagem Geração de Candidato & Teste

Segue a descrição de cada passo do Algoritmo Apriori:

* inicialmente, varrer todo o Banco de Dados (BD) uma vez em busca de itemsets de tamanho 1 que sejam frequentes
* Repita:
   * Gere itemsets de tamanho (k+1) dos itemsets frequentes de tamanho k
   * Teste os itemsets candidatos no BD para achar itemsets frequentes de tamanho (k+1)
   * faça k := k + 1
* até nenhum conjunto frequente ou candidato puder ser gerado
* retorne todos os itemsets frequentes derivados 

Vamos para uma exemplo concreto.Vamos definir o suporte mínimo como sendo 2 (*minsup = 2*) e que tenhamos um seguinte BD:

![](imgs/2.png)

O primeiro passo seria varrer todo o BD em busca dos itemsets de tamanho 1 que sejam frequentes. Então teríamos:

![](imgs/3.png)

Depois entramos num loop gerando candidatos de tamanho k+1 dos itemsetes frequentes de tamanho k:

![](imgs/4.png)

Fazemos o teste buscando itemsets frequentes aquivalentes aos itemsets candidatos gerados no passo anterior. Os que estão em azul não são frequentes e portanto são eliminados:

![](imgs/5.png)

![](imgs/6.png)

Com esses novos itemsets frequentes de tamanho 2, iremos seguir no loop e vamos gerar itemsets candidatos de tamanho K+1, ou seja, de tamanho 3. Como AB não é frequente, então ABC não será derivado, apenas BCE:

![](imgs/7.png)

Repetimos o processo de varrer o BD buscando itemsets frequentes aquivalentes aos itemsets candidatos gerados no passo anterior e verificamos que o candidato BCE é frequente:

![](imgs/8.png)

Terminamos o processo e então retornamos os itemsets frequentes encontrados.

Um dos passos fudamentais nesse algoritmo é a geração de candidatos. Como gera-los de maneira eficiente? Uma das maneiras seria seguir os passos listados abaixo: 

* fazer o auto-agrupamento (abc + bcd = abcd)
* podagem (quando um subset do auto-agrupamento não existe no BD)

Exemplo:

* $F_3 = {abc, abd, acd, ace, bcd}$
* auto-agrupamento:
   * *abcd* agrupamento que surge das transações *abc* e *abd*
   * *acde* agrupamento que surge das transações *acd* e *ace*
* podagem:
   * *acde* é removido/podado como candidato pois seu subset *ade* não existe em $F_3$


# Apriori: Melhorias e Alternativas

Encontrar o suporte para cada candidato é um processo custoso. Se você tem 10 candidatos, muito provavelmente você terá que varrer todo o BD 10 vezes para encontrar os respectivos suportes.

Seguem algumas abordagem sugeridas para diminuir o custo computacional do Algoritmo Apriori:

* **Reduzir os passos de varredura sobre o BD**:

    * Particionamento (e.g Saravage et al. 1995)
    * Contagem dinâmica de Itemsets (Brin et al. 1997)
    
* **Encolher número de candidatos**:

    * Hashing (e.g, DHP: Park et al, 1995)
    * Podagem por suporte do limear inferior (e.g Bayardo 1998) 
    * Amostragem (e.g Toivonen, 1996)
 
* **Exporar estrutura especiais de dados**:

    * Projeção de Árvore (Aggarwal et al, 2001)
    * Minerador-H (Pei, et al, 2001)
    * Decomposição por Hipercubo (e.g, LCM: Uno, et al, 2004)


### Particionamento: varrer o BD apenas duas vezes

Começamos por um teorema:

"*Qualquer itemset que é potencialmente frequente em um BD de transações (BDT) deve ser frequente em pelo menos uma das partições do  BDT.*"

O método de particionamento consiste em:

* Particionar o BD
* Encontar os itemsets frequentes para cada partição (padrões frequentes locais)
* Consolidar os padrões frequentes globais 

### Hashing Direto e Podagem (DHP)

O DHP (Direct Hashing and Pruning) tem por objetivo reduzir o número de candidatos. Primeiro precisamos entender o que é Hashing Direto e há um vídeo bem curto que explica muito bem a ideia por trás dessa abordagem, assista ele [aqui](https://www.youtube.com/watch?v=SwA_pQH0ihQ).

A ideia é criar uma tabela com buckets contendo itemsets de tamanho k (k-itemsets):

* Cada combinação de k-itemsets candidatos gerado é mapeado para um bucket da Tabela de Hashing Direto e assim a contagem desse bucket é incrementado. 

* Se um bucket, que um k-itemset está associado, não tiver contagem superior a suporte mínimo, o bucket em questão será "podado" e consequentemente os k-itemsets candidatos do bucket em questão não serão considerados frequentes.





### Explorando Dados de Formato Vertical: ECLAT

ECLAT é o acrônimo para "Equivalente Class Transformation" e esse algoritmo tenta explorar as vantagem de dados em formato vertical. 

Um BDT tem o formato horizontal como segue:

![](imgs/10.png)

Mas ele pode ser transformado para um formato vertical como segue:

![](imgs/11.png)

Qual a vantagem disso? Podemos pesquisar hiper-itemsets com base em seus sub-itemsets. Por exemplo:

* $t(e) = {T_{10}, T_{20}, T_{30}}$; 

* $t(a) = {T_{10}, T_{20}}$;

* $t(ae) = t(e) \cap t(a) = {T_{10}, T_{20}}$

Com isso podemos derivar padrões frequentes. Para acelerar ainda mais o processo de minerar padrões frequentes nesse tipo de estrutura de dados podemos monitorar as diferenças ao invés da interseção, pois o resultado da diferença é menor que a interseção de intemsets frequentes, assim salvamos um pouco de memória:

* $ t(ce) = {T_{10}, T_{20}}$

* $ difset(ce) = {T_{20}} $

### FPGrowth: Minerando Padrões Frequentes pelo Crescimento de Padrões

A ideia principal dessa abordagem é que *padrões frequentes crescem*, por isso do nome *Frequent Patterns Growth* (FPGrowth).

Seguem os passos desse algoritmo:

* Encontre itemsets frequentes de tamanho 1 e particione o BD em cada um de tais 1-itemsets frequentes.
* Recursivamente cresça os padrões frequentes aplicando o passo anterior para cada uma dos BDs partiçionados (também conhecidos como BDs particionados)
* Para facilitar a eficiência do processamento, uma estrura de dados eficiente chamada FP-tree pode ser usada (veja mais sobre FP-tree [aqui](https://dzone.com/articles/machinex-understanding-fp-tree-construction#:~:text=To%20put%20it%20simply%2C%20an,items%2C%20their%20paths%20may%20overlap.)).

Usando as FP-tree o processo de mineração de padrões seria o seguinte:

* Recursivamente construa e minere (condicionalmente) as FP-trees.
* Até a FP-tree estar vazia, ou até ela conter um path (paths únicos irão gerar todas suas possíveis combinações de sub-paths, cada um deles sendo padrões frequentes).

As imagens que usarei a seguir para exemplificar a construção de uma FP-trees eu retirei dos slides do curso [Pattern Discovery in Data Mining](https://www.coursera.org/learn/data-patterns).

Vamos supor que temos o seguinte BD de transações:

![](imgs/15.png)

Seguimos os seguintes passos:

* Varrer um BD uma vez para encontrar 1-itemsets frequentes (vamos usar 3 como suporte mínimo). Teremos então: f:4, a:3, c:4, b:3, m:3 e p:3.

* Ordene os itemsets frequentes em ordem decrescente: F-list = f-c-a-b-m-p.

* Varrer o BD novamente para construir FP-tree.

A seguir temos a nossa FP-tree do nosso exemplo de BDT. O 1-itemset *f* aparece 4 vezes, o 2-itemset *f-c*  aparece 3 vezes, o 3-itemset f-c-a aparace 3 vezes, o 4-itemset f-c-a-m aparece 2 vezes, 4-itemset f-c-a-b aparece 1 vez, o 5-itemset f-c-a-m-p aparece 2 vezes, o 5-itemset f-c-a-b-m aparece 1 vezes, etc.

![](imgs/16.png)

Podemos usar a abordagem recursiva de [Dividir e Conquistar](https://pt.wikipedia.org/wiki/Divis%C3%A3o_e_conquista) para construir essa árvore. Os padrões podem ser particionados de acordo o padrão corrente:
 
 * Padrões contendo *p*, ou seja, o BD condicionado a *p*: fcam:2, cb:1
 * Padrões contendo *m* mas não *p*, ou seja, BD condicionado a m:  fca:2, fcab:1
 * etc
 
Com isso podemos criar por exemplo uma base dos padrões: 
 
 ![](imgs/17.png)
 
Para os padrões p-condicionais podemos minerar os 1-itemsets frequentes. Por exemplo: 

* para a base p-condicional temos o *c*:3 1-itemset frequente pois *fcam*:2 e *cb*:1
* para a base m-condicional temos o 3-itemset *fca*:3 pois temos *fca*:2 e *fcab*=1
* etc

A vantagem dessa estrutura é que num único ramo dessa árvore, todos os seus sub-padrões frequentes podem ser gerados de forma imediata:

![](imgs/18.png)

O que fazer se a estrutura de dados FP-tree não couber na memória? Pode-se fazer a projeção num BD (DB Projection).

Para uma aplicação dessa abordagem usando Python, leia [Understand and Build FP-Growth Algorithm in Python](https://towardsdatascience.com/understand-and-build-fp-growth-algorithm-in-python-d8b989bab342)



# Exercícios

Exemplo de exercício:

![](imgs/19.png)

![](imgs/20.png)

![](imgs/21.png)

# Exercício de Programação

### Description

In this programming assignment, you are required to implement the Apriori algorithm and apply it to mine frequent itemsets from a real-life data set.

### Input

The provided input file ("categories.txt") consists of the category lists of 77,185 places in the US. Each line corresponds to the category list of one place, where the list consists of a number of category instances (e.g., hotels, restaurants, etc.) that are separated by semicolons.

An example line is provided below:


```Local Services;IT Services & Computer Repair```

In the example above, the corresponding place has two category instances: "Local Services" and "IT Services & Computer Repair".

```categories.txt```

### Output

You need to implement the Apriori algorithm and use it to mine category sets that are frequent in the input data. When implementing the Apriori algorithm, you may use any programming language you like. We only need your result pattern file, not your source code file.

After implementing the Apriori algorithm, please set the relative minimum support to 0.01 and run it on the 77,185 category lists. In other words, you need to extract all the category sets that have an absolute support larger than 771.

**Part 1**

Please output all the length-1 frequent categories with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category and should be in the following format:

```support:category```

For example, suppose a category (Fast Food) has an absolute support 3000, then the line corresponding to this frequent category set in "patterns.txt" should be:

```3000:Fast Food```

**Part 2**

Please write all the frequent category sets along with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category set and should be in the following format:


```support:category_1;category_2;category_3```


For example, suppose a category set (Fast Food; Restaurants) has an absolute support 2851, then the line corresponding to this frequent category set in "patterns.txt" should be:

```2851:Fast Food;Restaurants```

### Important Tips

Make sure that you format each line correctly in the output file. For instance, use a semicolon instead of another character to separate the categories for each frequent category set.

In the result pattern file, the order of the categories does not matter. For example, the following two cases will be considered equivalent by the grader:

**Case 1**:

```2851:Fast Food;Restaurants```

**Case 2**:

```2851:Restaurants;Fast Food```

### How to submit

When you're ready to submit, you can upload files for each part of the assignment on the "My submission" tab.

# Resposta

In [5]:
with open("data/categories.txt", "r") as file:
    categories = [line.rstrip().split(";") for line in file]
categories[:5]

[['Breakfast & Brunch', 'American (Traditional)', 'Restaurants'],
 ['Sandwiches', 'Restaurants'],
 ['Local Services', 'IT Services & Computer Repair'],
 ['Restaurants', 'Italian'],
 ['Food', 'Coffee & Tea']]

### Part 1

1-itemsets frequentes (suporte relativo 0.01)

A implementação a seguir foi criada por [Andrewngai](https://towardsdatascience.com/@andrewngai9255) e que é apresentada em seu artigo no medium intitulado [Understand and Build FP-Growth Algorithm in Python](https://towardsdatascience.com/understand-and-build-fp-growth-algorithm-in-python-d8b989bab342)

In [4]:
class node:
    def __init__(self, word, word_count=0, parent=None, link=None):
        self.word=word
        self.word_count=word_count
        self.parent=parent
        self.link=link
        self.children={}

#tree traversal
    def visittree(self):
#        if self is None:
#            return None
        output=[]
        output.append(str(vocabdic[self.word]) + " " +str(self.word_count))
        if len(list(self.children.keys()))>0:
            for i in (list(self.children.keys())):
                output.append(self.children[i].visittree())
        return output
  
              
'''      Build FPTREE class and method       '''        
class fptree:
    def __init__(self, data, minsup=400):
        #raw data and minminual support
        self.data=data
        self.minsup=minsup
        
        #null root
        self.root= node(word="Null", word_count=1)
        
        #each line of transaction with new order from the most frequent items to less
        self.wordlinesort=[]
        #node table containing link of all nodes of same word
        self.nodetable=[]
        #dictionary contaiing word more than the minsupport count with des order
        self.wordsortdic=[]
       
        #dictionaly containing word and the support count        
        self.worddic={}
        #dictionary with word and it's postion of the support count rank
        self.wordorderdic={}
#        
#        self.preprocess(data)
#        #first scan to build all the necessay dictionary
        self.construct(data)
        #second scan and build fp tree line  by line            
    def construct(self, data):
                #get support count for all word
        for tran in data:
            for words in tran:
                if words in self.worddic.keys():
                    self.worddic[words]+=1
                else:
                    self.worddic[words]=1
        wordlist = list(self.worddic.keys())
        #prune all the world with < min support count
        for word in wordlist:
            if(self.worddic[word]<self.minsup):
                del self.worddic[word]
        #sort the remaing items des, with first word count than work#id        
        self.wordsortdic = sorted(self.worddic.items(), key=lambda x: (-x[1],x[0])) 
        #create a table containing word, wordcount and all link node of that word
        t=0
        for i in self.wordsortdic:
            word = i[0]
            wordc = i[1]
            self.wordorderdic[word]=t
            t+=1
            wordinfo = {'wordn':word, 'wordcc':wordc, 'linknode': None}
            self.nodetable.append(wordinfo)
        #construct fptree line by line
    
        for line in data:
            supword=[]
            for word in line:
                #only keep words with support count higher than minsupport
                if word in self.worddic.keys():
                    supword.append(word)
           #insert words to the fp tree
            if len(supword)>0:
                #reorder the words 
                sortsupword = sorted(supword, key = lambda k: self.wordorderdic[k])
                self.wordlinesort.append(sortsupword)
                #enter the word one by one from begining
                R = self.root
#                print(sortsupword)
                for i in sortsupword:                  
                    if i in R.children.keys():
                        R.children[i].word_count +=1
                        R=R.children[i]
                    else:

                        R.children[i] = node(word=i,word_count=1,parent=R,link=None)
                        R=R.children[i]
                        # link this node to nodetable
                        for wordinfo in self.nodetable:
                            if wordinfo["wordn"] == R.word:
                                # find the last node of the  node linklist
                                if wordinfo["linknode"] is None:
                                    wordinfo["linknode"] = R
                                else:
                                    iter_node = wordinfo["linknode"]
                                    while(iter_node.link is not None):
                                        iter_node = iter_node.link
                                    iter_node.link = R

# create transactions for conditinal tree   
    def condtreetran(self,N):
        if N.parent is None:
            return None
        
        condtreeline =[]
        #starting from the leaf node reverse add word till hit root
        while N is not None:
            line=[]
            PN = N.parent
            while PN.parent is not None:
                line.append(PN.word)
                PN=PN.parent
            #reverse order the transaction
            line = line[::-1]
            for i in range(N.word_count):
                condtreeline.append(line)   
            #move on to next linknode
            N=N.link
        return condtreeline
    
#Find frequent word list by creating conditional tree
    def findfqt(self,parentnode=None):
        if len(list(self.root.children.keys()))==0:
            return None
        result=[]
        sup=self.minsup
        #starting from the end of nodetable
        revtable = self.nodetable[::-1]
        for n in revtable:
            fqset=[set(),0]
            if(parentnode==None):      
                fqset[0]={n['wordn'],}
            else:
                fqset[0] = {n['wordn']}.union(parentnode[0])
            fqset[1]=n['wordcc']
            result.append(fqset)
            condtran = self.condtreetran(n['linknode'])
            #recursively build the conditinal fp tree
            contree= fptree(condtran,sup)
            conwords = contree.findfqt(fqset)
            if conwords is not None:
                for words in conwords:
                    result.append(words)
        return result

#check if tree hight is larger than 1 
    def checkheight(self):
        if len(list(self.root.children.keys()))==0:
            return False
        else:
            return True

In [10]:
fp_tree = fptree(categories, len(categories)*0.01)

In [12]:
frequentitemset = fp_tree.findfqt() # mining frequent patterns
frequentitemset = sorted(frequentitemset, key = lambda k: -k[1])

In [15]:
frequentitemset

[[{'Restaurants'}, 25071],
 [{'Shopping'}, 11233],
 [{'Food'}, 9250],
 [{'Beauty & Spas'}, 6583],
 [{'Health & Medical'}, 5121],
 [{'Nightlife'}, 5088],
 [{'Home Services'}, 4785],
 [{'Bars'}, 4328],
 [{'Bars', 'Nightlife'}, 4328],
 [{'Automotive'}, 4208],
 [{'Local Services'}, 3468],
 [{'Active Life'}, 3103],
 [{'Fashion'}, 3078],
 [{'Fashion', 'Shopping'}, 3078],
 [{'Event Planning & Services'}, 2975],
 [{'Fast Food'}, 2851],
 [{'Fast Food', 'Restaurants'}, 2851],
 [{'Pizza'}, 2657],
 [{'Pizza', 'Restaurants'}, 2657],
 [{'Nightlife', 'Restaurants'}, 2533],
 [{'Mexican'}, 2515],
 [{'Mexican', 'Restaurants'}, 2515],
 [{'Hotels & Travel'}, 2495],
 [{'Bars', 'Restaurants'}, 2423],
 [{'Bars', 'Nightlife', 'Restaurants'}, 2423],
 [{'American (Traditional)'}, 2416],
 [{'American (Traditional)', 'Restaurants'}, 2416],
 [{'Sandwiches'}, 2364],
 [{'Restaurants', 'Sandwiches'}, 2364],
 [{'Arts & Entertainment'}, 2271],
 [{'Coffee & Tea'}, 2199],
 [{'Coffee & Tea', 'Food'}, 2199],
 [{'Food', 'Re

### Salvando o resultado no Formato Especificado no EP

In [23]:
# parte 1 do ep 
file = open("patterns.txt", "w")

for k_itemset in frequentitemset:
    if not len(k_itemset[0]) > 1:
        one_itemset =  str(k_itemset[1]) + ":" + ";".join(k_itemset[0]) + "\n"
        print(one_itemset)
        file.write(one_itemset)
    
        
file.close()

25071:Restaurants

11233:Shopping

9250:Food

6583:Beauty & Spas

5121:Health & Medical

5088:Nightlife

4785:Home Services

4328:Bars

4208:Automotive

3468:Local Services

3103:Active Life

3078:Fashion

2975:Event Planning & Services

2851:Fast Food

2657:Pizza

2515:Mexican

2495:Hotels & Travel

2416:American (Traditional)

2364:Sandwiches

2271:Arts & Entertainment

2199:Coffee & Tea

2091:Hair Salons

1848:Italian

1774:Burgers

1716:Auto Repair

1694:Doctors

1667:Nail Salons

1629:Chinese

1593:American (New)

1586:Home & Garden

1497:Pets

1442:Fitness & Instruction

1431:Hotels

1424:Real Estate

1424:Grocery

1369:Breakfast & Brunch

1195:Dentists

1150:Specialty Food

1138:Women's Clothing

1115:Bakeries

1025:Professional Services

1018:Ice Cream & Frozen Yogurt

1002:Cafes

875:Financial Services

874:Pubs

870:Pet Services

848:Japanese

823:General Dentistry

818:Sports Bars

798:Sushi Bars



In [27]:
# parte 1 do ep 
file = open("patterns.txt", "w")
for k_itemset in frequentitemset:
    freq_itemset =  str(k_itemset[1]) + ":" + ";".join(k_itemset[0]) + "\n"
    print(freq_itemset)
    file.write(freq_itemset)
file.close()

25071:Restaurants

11233:Shopping

9250:Food

6583:Beauty & Spas

5121:Health & Medical

5088:Nightlife

4785:Home Services

4328:Bars

4328:Bars;Nightlife

4208:Automotive

3468:Local Services

3103:Active Life

3078:Fashion

3078:Fashion;Shopping

2975:Event Planning & Services

2851:Fast Food

2851:Restaurants;Fast Food

2657:Pizza

2657:Restaurants;Pizza

2533:Restaurants;Nightlife

2515:Mexican

2515:Restaurants;Mexican

2495:Hotels & Travel

2423:Restaurants;Bars

2423:Restaurants;Bars;Nightlife

2416:American (Traditional)

2416:Restaurants;American (Traditional)

2364:Sandwiches

2364:Restaurants;Sandwiches

2271:Arts & Entertainment

2199:Coffee & Tea

2199:Food;Coffee & Tea

2101:Restaurants;Food

2091:Hair Salons

2091:Hair Salons;Beauty & Spas

1848:Italian

1848:Restaurants;Italian

1774:Burgers

1774:Restaurants;Burgers

1716:Auto Repair

1716:Auto Repair;Automotive

1694:Doctors

1694:Doctors;Health & Medical

1667:Nail Salons

1667:Nail Salons;Beauty & Spas

1629:Chines